# Домашнее задание 4
## Оценки
### Щербаков Игорь, СКБ 171
---

## 4.1 Проверка гипотез о виде распределения
## 4.1.1 Задание для рассматриваемых распределений

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter

In [2]:
def geom_pmf(x, p):
    return p*(1-p)**x

def generate_geom_sample(p = 0.5, sample_size = 100):
    res = []
    for i in range(sample_size):
        number=0;
        length=0;
        x=random.uniform(0,1)
        while(x>=length+geom_pmf(number, p)):
            length+=geom_pmf(number, p)
            number+=1
        res+=[number]
    return res

p=0.5
geom5=[]
geom10=[]
geom100=[]
geom1000=[]
geom100000=[]
for i in range(5):
    geom5+=[generate_geom_sample(p,5)]
for i in range(5):
    geom10+=[generate_geom_sample(p,10)]
for i in range(5):
    geom100+=[generate_geom_sample(p,100)]
for i in range(5):
    geom1000+=[generate_geom_sample(p,1000)]
for i in range(5):
    geom100000+=[generate_geom_sample(p,10000)]
    
geoms = [geom5,geom10,geom100,geom1000, geom100000]    
    
def geom_cdf(x,p=0.5):
    return 1-(1-p)**(x+1)    
    
def inverse_pareto_cdf(y, alpha, c0):
    if (1-y)**(1/alpha)==0:
        return 2.2250738585072014e-308
    return c0/((1-y))**(1/alpha)
def generate_pareto_sample(alpha, c0 = 1, sample_size = 10):
        result=[]
        for i in range(0,sample_size):
            x=random.uniform(0,1)
            #print(str(x))
            result+=[inverse_pareto_cdf(x, alpha, c0)]
        return result
    
alpha=2
c0=1
pareto5=[]
pareto10=[]
pareto100=[]
pareto1000=[]
pareto100000=[]
for i in range(5):
    pareto5+=[generate_pareto_sample(alpha, c0, 5)]
for i in range(5):
    pareto10+=[generate_pareto_sample(alpha, c0, 10)]
for i in range(5):
    pareto100+=[generate_pareto_sample(alpha, c0, 100)]
for i in range(5):
    pareto1000+=[generate_pareto_sample(alpha, c0, 1000)]
for i in range(5):
    pareto100000+=[generate_pareto_sample(alpha, c0, 100000)]
    
paretos = [pareto5,pareto10,pareto100,pareto1000, pareto100000]  

def pareto_pdf(x, alpha=2, c0=1):
    if x>=c0:
        return alpha/c0*(c0/x)**(alpha+1)
    else:
        return 0

### a) Геометрическое Распределение

### Критерий согласия хи-квадрат

Для выборки объемом $n$

Гипотеза $H_0$ - случайная величина подчиняется закону распределения геометрического распределения с параметром p = 0.5

##### Описание критерия

В $\chi^2$-критерии Пирсона используется мера расхождения $\chi^2=\sum\limits_{i=1}^{m}\dfrac{(v_i-np_i)^2}{np_i}$. Эта статистика имеет $\chi^2$ распределение с  $k = m - r - 1$ степенями свободы для сложных гипотез, $k = m -1$ для простых гипотез при $n\rightarrow \infty$, где $m$ - число интервалов эмперического наблюдения, $r$ - количество параметров теоритического распределения.  

Изначальное число интервалов для непрерывного распределения будем искать по формуле Старджесса: $ m = \left \lfloor{\log_2 n}\right \rfloor + 1$. Для дискретного примем за m количество различных элементов выборки.

Для применения критерия необходмо, чтобы в каждом интервале было не менее пяти значений. Если это не выполняется, стоит объединиять интервалы, в которых мало значений, с соседними. (В этой работе это не моделируется для дискретных)

Для выбранного уровня значимости по таблице распределения $\chi^2$ находится критическое значение $\chi^2_{\alpha; k}$. Если $\chi^2 > \chi^2_{\alpha;k}$, то принятая гипотеза $H_0$ отвергается, в противном случае принимается.

##### Свойства критерия

**Достоинство** данного критерия в том, что он универсален и подходит для исследования как дискретных, так и непрерывных величин. 


**Недостаток** заключается в зависимости от объема выборки, так как статистика имеет $\chi^2$-распределение только при $n\rightarrow \infty$.

##### Проверка соответствия выборки и распределения

In [3]:
from scipy.stats import chi2

In [11]:
def compute_quantile(cdf, a, eps=0.01, max_range=30):
    xx = np.arange(-1, max_range, eps)
    for x in xx:
        if cdf(x)<a and cdf(x+eps)>a:
            return x+eps

In [5]:
def chi_discrete(geom, p = 0.5):  #для выборки geom построить меру расхождения сhi^2
    chi = 0
    
    n = len(geom)
    do = dict(Counter(geom))
    #lists = sorted(do.items()) 
    m = max(max(do.keys())+1,5) #мы разбиваем хотя бы на 5 интервалов
    v = [0]*(m+1)
    for key in do.keys():
        v[key]+=do[key]
    #print(m)
    for i in range(0,m):
        pi = geom_pmf(i, p)
        chi += 1.0*(v[i] - n*pi)**2/(n*pi)
    return chi, m

In [6]:
def chi_results(dists, par=0.5, complex_hyp=False, chi_func=chi_discrete):
    alphas = [0.1, 0.05]
    for alpha in alphas:

        print("\n","+"*20,"Уровень значимости",alpha,"+"*20,"\n")
        
        
        for geom_of_size in dists:
            for no,geom in enumerate(geom_of_size):
                if(not complex_hyp):
                    chi, m = chi_func(geom, p=par)
                else:
                    n = len(geom)
                    p1 = 1/(n/(n-1)*np.mean(geom)+1)
                    chi, m = chi_discrete(geom, p = p1)
                chi = round(chi, 4)
                
                q = compute_quantile(chi2(m-1-complex_hyp).cdf, 1-alpha)
                q = round(q, 4)
                
                if(chi < q):
                    res = "Гипотеза принимается! ( V )"
                else:
                    res = "Гипотеза отклоняется! ( X )"
                if(complex_hyp):
                    print("Оценка параметра p =", round(p1,4))
                print("Выборка размера", len(geom), "  #"+str(no)+": \t","Хи-квадрат = "+str(chi),"Квантиль("+str(alpha)+";"+str(m-1-complex_hyp)+") = "+str(q)+"\t",res)
chi_results(geoms)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Хи-квадрат = 4.0438 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 Хи-квадрат = 3.6438 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 Хи-квадрат = 8.8438 Квантиль(0.1;4) = 7.78	 Гипотеза отклоняется! ( X )
Выборка размера 5   #3: 	 Хи-квадрат = 2.4438 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 Хи-квадрат = 1.6438 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #0: 	 Хи-квадрат = 13.0875 Квантиль(0.1;4) = 7.78	 Гипотеза отклоняется! ( X )
Выборка размера 10   #1: 	 Хи-квадрат = 2.6875 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #2: 	 Хи-квадрат = 2.0875 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #3: 	 Хи-квадрат = 4.2875 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 Хи-квадрат = 11.7

Точность метода возрастает с увеличением размера выборки. Попробуем проверить гипотезу с немного неверным параметром p = 0.54

In [7]:
chi_results(geoms,par=0.54)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Хи-квадрат = 4.2641 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 Хи-квадрат = 3.6249 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 Хи-квадрат = 11.3458 Квантиль(0.1;4) = 7.78	 Гипотеза отклоняется! ( X )
Выборка размера 5   #3: 	 Хи-квадрат = 2.8406 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 Хи-квадрат = 1.451 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #0: 	 Хи-квадрат = 18.1001 Квантиль(0.1;4) = 7.78	 Гипотеза отклоняется! ( X )
Выборка размера 10   #1: 	 Хи-квадрат = 2.381 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #2: 	 Хи-квадрат = 1.9864 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #3: 	 Хи-квадрат = 5.0008 Квантиль(0.1;4) = 7.78	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 Хи-квадрат = 18.52

Как и ожидалось теоретически, метод не дает надежных результатов на маленьких выборках, но работает верно при $n\rightarrow \infty$

### Критерий согласия хи-квадрат для сложной гипотезы

Для выборки объемом $n$

Гипотеза $H_0$ - случайная величина подчиняется закону распределения семейства геометрических распределений.

##### Описание критерия

Критерий применяется аналогично случаю с простой гипотезой, но в качестве параеметра берется его оценка макимального правдаподобия.

##### Свойства критерия

См. случай с простой гипотезой.

##### Проверка соответствия выборки и распределения

В предыдущей работе была получена ОМП $\hat{p} = \dfrac{1}{\frac{n}{n-1}\bar{x}+1} $. Используем ее в качестве параметра

In [8]:
chi_results(geoms,0.54, True)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Оценка параметра p = 0.4444
Выборка размера 5   #0: 	 Хи-квадрат = 3.9334 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.5714
Выборка размера 5   #1: 	 Хи-квадрат = 3.6777 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.3077
Выборка размера 5   #2: 	 Хи-квадрат = 4.0479 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.5
Выборка размера 5   #3: 	 Хи-квадрат = 2.4438 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.6667
Выборка размера 5   #4: 	 Хи-квадрат = 1.2794 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.3333
Выборка размера 10   #0: 	 Хи-квадрат = 3.9831 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.6429
Выборка размера 10   #1: 	 Хи-квадрат = 2.7211 Квантиль(0.1;3) = 6.26	 Гипотеза принимается! ( V )
Оценка параметра p = 0.6
Выборка размера 10   #2: 

ОМП дает дает значение параметра, хорошо подходящее под выборку

### б) Распределение Парето


### Критерий согласия Колмогорова

Для выборки объемом $n$

Гипотеза $H_0$ - случайная величина подчиняется закону распределения Парето с параметрами $\alpha = 2 ,c_0 = 1$

##### Описание критерия

В качестве меры расхождения используется максимум абсолютного значения разности Э.Ф.Р. и Т.Ф.Р $D_n$.

Таким образом, $D_n=max\left|F_n(x)-F(x)\right|$ - статистика критерия Колмогорова.

При $n\rightarrow\infty$ $P(D_n\sqrt{n} \geqslant \lambda)$ стремится к распределению Колмогорова $ P(\lambda)=1 - \sum\limits_{k=-\infty}^{+\infty}(-1)^ke^{-2k^2\lambda^2}$.

Необходимо вычислить $\lambda=D_n\sqrt{n}$ и сравнить с $\lambda_\alpha$, где $\alpha$ - уровень значимости. Если $D_n\sqrt{n} \leq \lambda_\alpha$, то гипотеза принимается.

Вместо $D_n \sqrt{n}$ лучше использовать вид статистики с поправкой Большева: $S=\dfrac{6nD_n+1}{6\sqrt{n}}$, т.к. к распределению Колмогорова он сходится быстрее.



##### Свойства критерия

**Достоинство** данного критерия в том, что он может применяться на выборках любых объемов. 

**Недостаток** - возможность использования только для непрерывных распределений.

##### Проверка соответствия выборки и распределения

In [9]:
from scipy.special import kolmogorov

In [35]:
def find_next(i, j ,n, m, list1, list2):
    new=-1
    if i<n and j<m:
        if list1[i] < list2[j]:
            new = list1[i]
            i += 1
        else:
            new = list2[j]
            j += 1
    elif i < n:
        new = list1[i]
        i += 1
    elif j < m:
        new = list2[j]
        j += 1
    while i <n and list1[i] == new:
        i += 1
    while j < m and list2[j] == new:
        j += 1
    return (i,j)
def diff(var1, var2): #находит по вариационным рядам разницу между функицями распределения
    i, j = 0, 0
    list1 =  var1
    list2 =  var2
    n,m = len(list1), len(list2)
    ret = 0
    while (i, j) != (n, m):
        ret = max(ret, abs(i/(n)-j/(m)))
        i,j = find_next(i, j, n, m, list1, list2)
    return ret

Сгенерируем "идеальную" Функцию распределения Парето

In [57]:
def pareto_cdf(x, alpha=2, c0=1): 
    return 1-(c0/x)**alpha

def emp_cdf(x, data):
    return data[data<=x].size / data.size


In [79]:
def kolmogorov_res(dists,par1=2, par2=1, complex_hyp=False):
    alphas = [0.1, 0.05]
    for alpha in alphas:
        print("\n","+"*20,"Уровень значимости",alpha,"+"*20,"\n")
        
        
        for pareto_of_size in dists:
            for no,pareto in enumerate(pareto_of_size):
                pareto = np.array(pareto)
                volume = len(pareto)
                F_n = lambda x: emp_cdf(x, pareto)
                F = lambda x: pareto_cdf(x, p1, par2)
                if(not complex_hyp):
                    F = lambda x: pareto_cdf(x, par1, par2)
                else:
                    p1 = (volume-1)/sum(np.log(np.array(pareto)))
                    
                xx = np.arange(1,16, 0.01)
                #for x in xx:
                #    print(F(x), F_n(x))
                D_n = max(map(lambda x : abs(F_n(x) - F(x)), xx))

                S_n = np.sqrt(volume)*D_n+1/(6*np.sqrt(volume))
                S_n = round(S_n,4)
                K = lambda x: 1 - kolmogorov(x)
                q = compute_quantile(K, 1-alpha)
                q = round(q,4)
                if(S_n<=q):
                    res = "Гипотеза принимается! ( V )"
                else:
                    res = "Гипотеза отклоняется! ( X )"
                if(complex_hyp):
                    print("Оценка параметра alpha =", round(p1,4))
                print("Выборка размера", volume, "  #"+str(no)+": \t","S = "+str(S_n),"Квантиль = "+str(q)+"\t",res)
kolmogorov_res(paretos)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 S = 0.8959 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 S = 0.7846 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 S = 0.5008 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #3: 	 S = 0.5993 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 S = 1.0519 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #0: 	 S = 0.7179 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #1: 	 S = 0.8025 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #2: 	 S = 0.7828 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #3: 	 S = 0.9414 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 S = 1.1379 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 100   #0: 	 S = 0.7001 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 100   

Точность метода возрастает с увеличением размера выборки. Попробуем проверить гипотезу с неверным параметром alpha = 3

In [73]:
kolmogorov_res(paretos, 3)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 S = 0.5723 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 S = 0.4599 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 S = 0.4868 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #3: 	 S = 0.9129 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 S = 1.3746 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборка размера 10   #0: 	 S = 0.7473 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #1: 	 S = 1.2588 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборка размера 10   #2: 	 S = 1.2502 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборка размера 10   #3: 	 S = 0.7731 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 S = 0.8474 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборка размера 100   #0: 	 S = 2.1786 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборка размера 100   

Критерий работает

### Критерий согласия хи-квадрат

Для выборки объемом $n$

Гипотеза $H_0$ - случайная величина подчиняется закону распределения Парето с параметрами $\alpha = 2, c_0 = 1$

##### Проверка соответствия выборки и распределения

In [22]:
import scipy.integrate as integrate
import scipy.special as special
def chi_continuous(pareto, alpha = 3, c0 = 1):  #для выборки geom построить меру расхождения сhi^2
    chi = 0
    n = len(pareto)
    m = int(max(np.floor(np.log2(n)) , 5)) #мы разбиваем хотя бы на 5 интервалов
    v = [0]*(m)
    p = [0]*(m)
    mp = max(pareto)
    interval = (mp)/m

    for i in pareto:
        k = int((i)/interval)
        #print(max(pareto)-1,i, k)
        if(i!=mp):
            v[k]+=1
    v[m-1]+=1
    for i in range(0,m):
        ans, err = integrate.quad(lambda x: pareto_pdf(x,alpha,c0),i*interval,(i+1)*interval)
        p[i]+= ans

    
    def correct_p_v(p, v, n):
        for i in range(len(v)):
            while (i < len(v)) and v[i] < 5:
                if (i+1) != len(v):
                    v[i] += v[i+1]
                    p[i] += p[i+1]
                    del v[i+1], p[i+1]
                else:
                    v[i-1] += v[i]
                    p[i-1] += p[i]
                    del v[i], p[i]
        return p, v, n
    p, v, n = correct_p_v(p, v, n)

    for i in range(0,len(v)):
        pi = p[i]
        #print(v[i], n*p[i])
        chi += 1.0*(v[i] - n*pi)**2/(n*pi)
    return chi, len(v)+2 

In [40]:
def chi_results2(dists, par1=2, par2=1, complex_hyp=False, chi_func=chi_continuous):
    alphas = [0.1, 0.05]
    for alpha in alphas:

        print("\n","+"*20,"Уровень значимости",alpha,"+"*20,"\n")
        
        
        for pareto_of_size in dists:
            for no,pareto in enumerate(pareto_of_size):
                if(not complex_hyp):
                    chi, m = chi_func(pareto, par1, par2)
                else:
                    n = len(pareto)
                    p1 = (n-1)/sum(np.log(np.array(pareto)))
                    chi, m = chi_continuous(pareto, p1, par2)
                chi = round(chi, 4)
                
                q = compute_quantile(chi2(m-1-complex_hyp).cdf, 1-alpha)
                q = round(q, 4)
                
                if(chi < q):
                    res = "Гипотеза принимается! ( V )"
                else:
                    res = "Гипотеза отклоняется! ( X )"
                if(complex_hyp):
                    print("Оценка параметра p =", round(p1,4))
                print("Выборка размера", len(pareto), "  #"+str(no)+": \t","Хи-квадрат = "+str(chi),"Квантиль("+str(alpha)+";"+str(m-1-complex_hyp)+") = "+str(q)+"\t",res)
chi_results2(paretos)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Хи-квадрат = 1.0805 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 Хи-квадрат = 0.1007 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 Хи-квадрат = 0.1063 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #3: 	 Хи-квадрат = 0.0203 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 Хи-квадрат = 0.3476 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #0: 	 Хи-квадрат = 0.025 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #1: 	 Хи-квадрат = 0.0084 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #2: 	 Хи-квадрат = 0.0574 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #3: 	 Хи-квадрат = 0.4036 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 Хи-квадрат = 0.3241

Точность метода возрастает с увеличением размера выборки. Попробуем проверить гипотезу с неверным параметром alpha = 3

In [41]:
chi_results2(paretos, 3, 1)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Хи-квадрат = 0.3244 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #1: 	 Хи-квадрат = 0.0121 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #2: 	 Хи-квадрат = 0.0131 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #3: 	 Хи-квадрат = 0.0012 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 5   #4: 	 Хи-квадрат = 0.0695 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #0: 	 Хи-квадрат = 0.0012 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #1: 	 Хи-квадрат = 0.0002 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #2: 	 Хи-квадрат = 0.004 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #3: 	 Хи-квадрат = 0.0651 Квантиль(0.1;2) = 4.61	 Гипотеза принимается! ( V )
Выборка размера 10   #4: 	 Хи-квадрат = 0.0477

Как и ожидалось теоретически, метод не дает надежных результатов на маленьких выборках, но работает верно при $n\rightarrow \infty$

### Критерий согласия Колмогорова для сложной гипотезы

Гипотеза $H_0$ - случайная величина подчиняется закону распределения семейства распределений Парето с параметром $c_0 = 1$.

##### Описание критерия

Критерий применяется аналогично случаю с простой гипотезой, но в качестве параеметра берется его оценка макимального правдаподобия.

##### Свойства критерия

См. случай с простой гипотезой.

##### Проверка соответствия выборки и распределения

В предыдущей работе была получена ОМП $\hat{\alpha} = \dfrac{n-1}{\sum\limits_{i=1}^n \ln x_i} $. Используем ее в качестве параметра

In [80]:
kolmogorov_res(paretos, complex_hyp=True)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Оценка параметра alpha = 3.0112
Выборка размера 5   #0: 	 S = 0.5695 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 2.3373
Выборка размера 5   #1: 	 S = 0.663 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 2.1121
Выборка размера 5   #2: 	 S = 0.4752 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 1.4471
Выборка размера 5   #3: 	 S = 0.6141 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 1.4767
Выборка размера 5   #4: 	 S = 0.8327 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 1.859
Выборка размера 10   #0: 	 S = 0.7863 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 1.3562
Выборка размера 10   #1: 	 S = 0.4015 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра alpha = 1.8136
Выборка размера 10   #2: 	 S = 0.6743 Квантиль = 1.23	 Гипотеза принимается! ( V )
Оценка параметра a

ОМП дает дает значение параметра, хорошо подходящее под выборку

### Критерий согласия хи-квадрат для сложной гипотезы

Гипотеза $H_0$ - случайная величина подчиняется закону распределения семейства распределений Парето с параметром $c_0 = 1$.

##### Проверка соответствия выборки и распределения

В предыдущей работе была получена ОМП $\hat{\alpha} = \dfrac{n-1}{\sum\limits_{i=1}^n \ln x_i} $. Используем ее в качестве параметра

In [24]:
chi_results2(paretos, complex_hyp=1)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Оценка параметра p = 3.0112
Выборка размера 5   #0: 	 Хи-квадрат = 1.0805 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 2.3373
Выборка размера 5   #1: 	 Хи-квадрат = 0.1007 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 2.1121
Выборка размера 5   #2: 	 Хи-квадрат = 0.1063 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 1.4471
Выборка размера 5   #3: 	 Хи-квадрат = 0.0203 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 1.4767
Выборка размера 5   #4: 	 Хи-квадрат = 0.3476 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 1.859
Выборка размера 10   #0: 	 Хи-квадрат = 0.025 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 1.3562
Выборка размера 10   #1: 	 Хи-квадрат = 0.0084 Квантиль(0.1;1) = 2.71	 Гипотеза принимается! ( V )
Оценка параметра p = 1.8136
Выборка размера 10   

ОМП дает дает значение параметра, хорошо подходящее под выборку

## 4.1.1 Критерий однородности Смирнова

Для двух выборок:

Гипотеза $H_0$ - две выборки имеют одинаковые функции распределения

##### Описание критерия

В качестве меры совпадения распределений выборок используется статистка

$\lambda=\sqrt{\dfrac{n_1n_2}{n_1+n_2}} max\left| F_{n_1} - F_{n_2}\right|$

$n_1$, $n_2$ - объемы двух сравниваемых выборок,
$F_{n_1},\; F_{n_2}$ - Э.Ф.Р. выборок

По теореме Смирнова $\lambda$ стремится при $n, m \rightarrow \infty$ к распределению Колмогорова

Если $\lambda_\alpha$ - табличное критическое значение статистики, то гипотеза $H_0$ принимается, когда $\lambda ≤ \lambda_\alpha$.

Критерий используется для непрерывных распределений


##### Проверка совпадения распределений

In [15]:
flatpareto = []
for i in paretos:
    for j in i:
        flatpareto+=[sorted(j)]
paretocombs = list(itertools.combinations(flatpareto, 2))
pareto_comb_inds = list((i,j) for ((i,_),(j,_)) in itertools.combinations(enumerate(flatpareto), 2))

In [16]:
alphas = [0.1, 0.05]
sizes=[5, 10, 100, 1000, 100000]
for alpha in alphas:
    print("\n","+"*20,"Уровень значимости",alpha,"+"*20,"\n")
    for i,comb in enumerate(paretocombs):
        lam = np.sqrt(len(comb[0])*len(comb[1])/(len(comb[0])+len(comb[1])))*diff(*comb)
        lam = round(lam, 4)
        K = lambda x: 1 - kolmogorov(x)
        q = compute_quantile(K, 1-alpha)
        q = round(q, 4)
        if(lam <= q):
            res = "Гипотеза принимается! ( V )"
        else:
            res = "Гипотеза отклоняется! ( X )"
        print("Выборки n1 = "+str(len(comb[0]))+"  #"+str(pareto_comb_inds[i][0] % 5)," и ","n2 = "+str(len(comb[1]))+" #"+str(pareto_comb_inds[i][1] % 5),":")
        print("Лямбда = "+str(lam),"Квантиль = "+str(q)+"\t",res)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборки n1 = 5  #0  и  n2 = 5 #1 :
Лямбда = 0.3162 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 5 #2 :
Лямбда = 0.3162 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 5 #3 :
Лямбда = 0.6325 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 5 #4 :
Лямбда = 1.2649 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборки n1 = 5  #0  и  n2 = 10 #0 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 10 #1 :
Лямбда = 1.0954 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 10 #2 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 10 #3 :
Лямбда = 0.3651 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 10 #4 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #0  и  n2 = 100 #0 :
Лямбда = 0.9383 Квантиль 

Выборки n1 = 5  #3  и  n2 = 100000 #1 :
Лямбда = 0.5427 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #3  и  n2 = 100000 #2 :
Лямбда = 0.5417 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #3  и  n2 = 100000 #3 :
Лямбда = 0.541 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #3  и  n2 = 100000 #4 :
Лямбда = 0.5374 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #4  и  n2 = 10 #0 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #4  и  n2 = 10 #1 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #4  и  n2 = 10 #2 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #4  и  n2 = 10 #3 :
Лямбда = 1.278 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборки n1 = 5  #4  и  n2 = 10 #4 :
Лямбда = 0.7303 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  #4  и  n2 = 100 #0 :
Лямбда = 0.851 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 5  

Выборки n1 = 10  #3  и  n2 = 100000 #1 :
Лямбда = 0.8974 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #3  и  n2 = 100000 #2 :
Лямбда = 0.9049 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #3  и  n2 = 100000 #3 :
Лямбда = 0.8967 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #3  и  n2 = 100000 #4 :
Лямбда = 0.9109 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #4  и  n2 = 100 #0 :
Лямбда = 1.206 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #4  и  n2 = 100 #1 :
Лямбда = 1.1457 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #4  и  n2 = 100 #2 :
Лямбда = 1.1759 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #4  и  n2 = 100 #3 :
Лямбда = 1.2663 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборки n1 = 10  #4  и  n2 = 100 #4 :
Лямбда = 1.1759 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 10  #4  и  n2 = 1000 #0 :
Лямбда = 1.0919 Квантиль = 1.23	 Гипотеза принимается! ( V 

Выборки n1 = 1000  #0  и  n2 = 100000 #0 :
Лямбда = 0.5186 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #0  и  n2 = 100000 #1 :
Лямбда = 0.5979 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #0  и  n2 = 100000 #2 :
Лямбда = 0.5667 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #0  и  n2 = 100000 #3 :
Лямбда = 0.5922 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #0  и  n2 = 100000 #4 :
Лямбда = 0.4802 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #1  и  n2 = 1000 #2 :
Лямбда = 1.5429 Квантиль = 1.23	 Гипотеза отклоняется! ( X )
Выборки n1 = 1000  #1  и  n2 = 1000 #3 :
Лямбда = 0.6708 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #1  и  n2 = 1000 #4 :
Лямбда = 0.6708 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #1  и  n2 = 100000 #0 :
Лямбда = 1.0478 Квантиль = 1.23	 Гипотеза принимается! ( V )
Выборки n1 = 1000  #1  и  n2 = 100000 #1 :
Лямбда = 0.9449 Квантиль = 

Выборки n1 = 5  #1  и  n2 = 100000 #1 :
Лямбда = 0.7122 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #1  и  n2 = 100000 #2 :
Лямбда = 0.7151 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #1  и  n2 = 100000 #3 :
Лямбда = 0.7144 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #1  и  n2 = 100000 #4 :
Лямбда = 0.7197 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 5 #3 :
Лямбда = 0.3162 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 5 #4 :
Лямбда = 0.9487 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 10 #0 :
Лямбда = 0.3651 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 10 #1 :
Лямбда = 0.7303 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 10 #2 :
Лямбда = 0.5477 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  #2  и  n2 = 10 #3 :
Лямбда = 0.3651 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 5  

Выборки n1 = 10  #0  и  n2 = 100000 #4 :
Лямбда = 0.6844 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 10 #2 :
Лямбда = 0.6708 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 10 #3 :
Лямбда = 1.118 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 10 #4 :
Лямбда = 1.118 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 100 #0 :
Лямбда = 0.603 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 100 #1 :
Лямбда = 0.7538 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 100 #2 :
Лямбда = 0.5126 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 100 #3 :
Лямбда = 0.4824 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 100 #4 :
Лямбда = 0.6633 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 10  #1  и  n2 = 1000 #0 :
Лямбда = 0.7363 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 =

Выборки n1 = 100  #1  и  n2 = 100000 #2 :
Лямбда = 0.6671 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #1  и  n2 = 100000 #3 :
Лямбда = 0.6729 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #1  и  n2 = 100000 #4 :
Лямбда = 0.6573 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 100 #3 :
Лямбда = 0.5657 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 100 #4 :
Лямбда = 0.7071 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 1000 #0 :
Лямбда = 1.0107 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 1000 #1 :
Лямбда = 1.1918 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 1000 #2 :
Лямбда = 0.82 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 1000 #3 :
Лямбда = 1.3158 Квантиль = 1.36	 Гипотеза принимается! ( V )
Выборки n1 = 100  #2  и  n2 = 1000 #4 :
Лямбда = 1.1156 Квантиль = 1.36	 Гипотеза принима

Выборки n1 = 100000  #3  и  n2 = 100000 #4 :
Лямбда = 1.4154 Квантиль = 1.36	 Гипотеза отклоняется! ( X )


## 4.2 Проверка параметрических гипотез

## 4.2.1 Выбор данных

Возьмем сгененрированные данные из геометрического распределения c параметром $p = 0.5$, а именно те же выборки, что мы использовали ранее.

Для контроля сгененрируем другой набор выборок распределенных c параметром $p = 0.7$

$\xi \sim Geom(p)$

In [47]:
p=0.7
geom5=[]
geom10=[]
geom100=[]
geom1000=[]
geom100000=[]
for i in range(5):
    geom5+=[generate_geom_sample(p,5)]
for i in range(5):
    geom10+=[generate_geom_sample(p,10)]
for i in range(5):
    geom100+=[generate_geom_sample(p,100)]
for i in range(5):
    geom1000+=[generate_geom_sample(p,1000)]
for i in range(5):
    geom100000+=[generate_geom_sample(p,10000)]
    
geoms2 = [geom5,geom10,geom100,geom1000, geom100000]    

## 4.2.2 Постановка задачи

Мы рассматриваем параметрические гипотезы $H_0$ и $H_1$. Считая вид распределения известным, мы хотим определить, какой из параметров $\theta_0$ (гипотеза $H_0$) или  $\theta_1$ (гипотеза $H_1$).


Критерий - это правило, по которому конкретную выборку стоит отнести к распределению соответствующему $H_0$ или $H_1$. Таким образом, она делит выборочное пространство $\mathfrak{X}$ на непересекающиеся множества $\mathfrak{X_0}$ и $\mathfrak{X_1}$ неком отношении.

В нашем случае случае

$H_0: p = p_0 = 0.5$

$H_1: p = p_1 = 0.7$

Ошибкой I рода $\alpha$ называют отвержение $H_0$, когда она истинна. Ошибкой II рода $\beta$ - принятие $H_0$ за истинную, когда она ложна.

Т.к. мы используем синтетические данные, в качестве пояснения ошибок I и II в нашем примере напишем их определения.

Для простых гипотез:

$\alpha = P(x \in \mathfrak{X_{1}} | H_0)$

$\beta = P(x \in \mathfrak{X_{0}}|H_1) = 1 - P(x \in \mathfrak{X_{1\alpha}}|H_1)$.

## 4.2.3 Вычисление функции отношения правдоподобия

Функция отношения правдободобий:

$l(x)=\dfrac{L(X,\theta_1)}{L(X,\theta_0)}$

$H_0$ принимается если $l(x)<c$  

В нашем случае:

$l(x) = \dfrac{L(X,p_1)}{L(X, p_0)}$

$L(X, p_1) = \prod\limits_{i=1}^{n}p(1-p)^{x_i}=p^n \prod\limits_{i=1}^{n}(1-p)^{x_i}$

## 4.2.4 Вычисление количества материала

$\ln L(X, p_1) = n\ln p_1 + \ln (1-p_1) \sum\limits_{i=1}^n x_i$

$\ln (l(x)) = n \ln\left(\dfrac{p_1}{p_0}\right) + \ln\dfrac{1-p_1}{1-p_0} \sum\limits_{i=1}^n x_i \geq c' = \ln c$ 

$\sum\limits_{i=1}^n x_i \geq \dfrac{c' - n \ln\left(\dfrac{p_1}{p_0}\right)}{ \ln\dfrac{1-p_1}{1-p_0} }$

При $n\rightarrow\infty ,
\dfrac{Sn-\mu n}{\sigma \sqrt{n}}\rightarrow N(0,1)$

где $Sn=\sum_{i=1}^n x_i$ по Центральной Предельной Теореме
$\mu,\;\sigma^2$ - мат.ожидание и дисперсия.

Для геометрического распределения

$E\xi = \dfrac{1-p}{p}$

$D\xi = \dfrac{1-p}{p^2}$

Тогда для геометрического распределения в условиях гипотезы $H_0$:

$\dfrac{\sum\limits_{i=1}^n x_i - n\dfrac{1-p_0}{p_0}}{\sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)}} \sim N(0,1)$


$\dfrac{\sum\limits_{i=1}^n x_i - n\dfrac{1-p_0}{p_0}}{\sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)}}$ $ \geq \dfrac{\left(\dfrac{c' - n \ln\left(\dfrac{p_1}{p_0}\right)}{ \ln\dfrac{1-p_1}{1-p_0} }\right) - n\dfrac{1-p_0}{p_0}}{\sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)}}$

Тогда, $H_0$ принимается, если:

t = $\dfrac{\sum\limits_{i=1}^n x_i - n\dfrac{1-p_0}{p_0}}{\sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)}} < t_\alpha$

Где $t_\alpha$ - квантиль нормального распределения уровня $\alpha$

По тому, как мы строим наши гипотезы (либо параметр $p_0$, либо $p_1$), если отвергается $H_0$, мы выбираем $H_1$

Переходя к нормальному распределнию, мы попадаем в область непрерывных распределений мы уже не можем получить точное равенство квантилю. Если бы мы не перешли к нормальному распределению, нужно было бы в случае равенства "подбросить монетку" 

In [54]:
from scipy.stats import norm
def compute_t(x, p0):
    n = len(x)
    res = (np.sum(x) - n*(1-p0)/p0)/(np.sqrt(n*(1-p0)/p0**2))
    return abs(res)

quant = {0.05: 1.645, 0.1: 1.282}

def t_results(dists, par=0.5):
    alphas = [0.1, 0.05]
    for alpha in alphas:

        print("\n","+"*20,"Уровень значимости",alpha,"+"*20,"\n")
        
        
        for geom_of_size in dists:
            for no,geom in enumerate(geom_of_size):

                t = compute_t(geom, par)
                t = round(t, 4)
                
                q = quant[alpha]
                
                if(t < q):
                    res = "Гипотеза H0 принимается! ( H0 )"
                else:
                    res = "Гипотеза H0 отклоняется! ( H1 )"
                print("Выборка размера", len(geom), "  #"+str(no)+": \t","Cтатистика = "+str(t),"t("+str(alpha)+") = "+str(q)+"\t",res)
                
    

In [55]:
t_results(geoms, 0.5)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #1: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #2: 	 Cтатистика = 0.6325 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #3: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #4: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #0: 	 Cтатистика = 2.2361 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #1: 	 Cтатистика = 0.4472 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #2: 	 Cтатистика = 1.118 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #3: 	 Cтатистика = 0.2236 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #4: 	 Cтатистика = 0.8944 t(0.1) = 1.282	 Гипотеза H0 принима

Проверим для неправильного параметра p = 0.7

In [56]:
t_results(geoms, 0.7)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Cтатистика = 3.3476 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 5   #1: 	 Cтатистика = 3.3476 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 5   #2: 	 Cтатистика = 2.7761 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 5   #3: 	 Cтатистика = 0.0816 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #4: 	 Cтатистика = 0.0816 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #0: 	 Cтатистика = 6.3509 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #1: 	 Cтатистика = 3.1177 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #2: 	 Cтатистика = 0.2887 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #3: 	 Cтатистика = 1.9053 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #4: 	 Cтатистика = 0.6928 t(0.1) = 1.282	 Гипотеза H0 приним

Выразим $\sum\limits_{i=1}^n x_i < t_\alpha \sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)} + n\dfrac{1-p_0}{p_0}$ и аналогично найдем статистику при условии $H_1$

$\dfrac{\sum\limits_{i=1}^n x_i - n\dfrac{1-p_1}{p_1}}{\sqrt{n\left(\dfrac{1-p_1}{p_1^2}\right)}} < \dfrac{\left(t_\alpha \sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)} + n\dfrac{1-p_0}{p_0}\right) -  n\dfrac{1-p_1}{p_1}}{\sqrt{n\left(\dfrac{1-p_1}{p_1^2}\right)}}= t_{\beta}$

$t_{\beta} = \dfrac{\left(t_\alpha \sqrt{n\left(\dfrac{1-p_0}{p_0^2}\right)} + n\dfrac{1-p_0}{p_0}\right) -  n\dfrac{1-p_1}{p_1}}{\sqrt{n\left(\dfrac{1-p_1}{p_1^2}\right)}} = t_\alpha \sqrt{\dfrac{(1-p_0)p_1^2}{p_0^2 (1-p_1)}} - \dfrac{\sqrt{n}\left(\dfrac{1-p_0}{p_0}-\dfrac{1-p_1}{p_1}\right)}{\sqrt{\left(\dfrac{1-p_1}{p_1^2}\right)}}$

$\left(t_\alpha \sqrt{\dfrac{(1-p_0)p_1^2}{p_0^2 (1-p_1)}} - t_{\beta}\right)\sqrt{\left(\dfrac{1-p_1}{p_1^2}\right)} = \sqrt{n}\left(\dfrac{1-p_0}{p_0}-\dfrac{1-p_1}{p_1}\right)$

$\left\lceil\dfrac{ \left(t_\alpha \sqrt{\dfrac{(1-p_0)p_1^2}{p_0^2 (1-p_1)}} - t_{\beta}\right)^2 \left(\dfrac{1-p_1}{p_1^2}\right) }{\left(\dfrac{1-p_0}{p_0}-\dfrac{1-p_1}{p_1}\right)^2}\right\rceil = n $

Теперь, фиксируя $\alpha$ и $\beta$ мы можем найти n.

In [111]:
def get_n(p0,p1,alpha, beta):
    res = (quant[alpha]*np.sqrt( (1-p0)*p1**2/p0**2/(1-p1) )-quant[beta])**2 * ((1-p1)/p1**2)/ ((1-p0)/p0 - (1-p1)/p1)**2
    res = np.ceil(res)
    return res

По идее, если мы поменяем вероятности гипотез и ошибки I и II рода, полученный размер метериала не изменится. Это действительно так

In [110]:
print(get_n(p0=0.5, p1=0.3, alpha=0.05, beta=0.1) == get_n(p0=0.3, p1=0.5, alpha=0.1, beta=0.05))
print(get_n(p0=0.2, p1=0.3, alpha=0.05, beta=0.1) == get_n(p0=0.3, p1=0.2, alpha=0.1, beta=0.05))
print(get_n(p0=0.9, p1=0.3, alpha=0.05, beta=0.1) == get_n(p0=0.3, p1=0.9, alpha=0.1, beta=0.05))
print(get_n(p0=0.7, p1=0.9, alpha=0.05, beta=0.1) == get_n(p0=0.9, p1=0.7, alpha=0.1, beta=0.05))

True
True
True
True


Чем ближе параметры, тем труднее различить гипотезы и нужно большее n

In [117]:
print(get_n(p0=0.4, p1=0.5, alpha=0.05, beta=0.1))
print(get_n(p0=0.4, p1=0.41, alpha=0.05, beta=0.1))
print(get_n(p0=0.4, p1=0.401, alpha=0.05, beta=0.1))

8.0
166.0
13014.0


In [112]:
print(get_n(p0=0.5, p1=0.7, alpha=0.05, beta=0.1))

6.0


Мы получаем, что для вероятного различения гипотез при $\alpha = 0.05, \beta = 0.1$ нужно взять выборку размером 6. Действительно, начиная с размера выборок 10, правильные ответы преобладают над ошибками и мы можем сделать правильный вывод.

Проверим это для выборки с параметром 0.7

In [103]:
get_n(p0=0.7, p1=0.5, alpha=0.05, beta=0.1)

1.0

Получается размер выборки всего в 1 элемент, проверим

In [93]:
t_results(geoms2, 0.7)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Cтатистика = 0.4899 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #1: 	 Cтатистика = 1.2247 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #2: 	 Cтатистика = 0.0816 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #3: 	 Cтатистика = 0.4899 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #4: 	 Cтатистика = 0.0816 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #0: 	 Cтатистика = 0.5196 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #1: 	 Cтатистика = 0.1155 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #2: 	 Cтатистика = 1.3279 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #3: 	 Cтатистика = 0.9238 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #4: 	 Cтатистика = 0.5196 t(0.1) = 1.282	 Гипотеза H0 приним

In [60]:
t_results(geoms2, 0.5)


 ++++++++++++++++++++ Уровень значимости 0.1 ++++++++++++++++++++ 

Выборка размера 5   #0: 	 Cтатистика = 0.6325 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #1: 	 Cтатистика = 1.5811 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 5   #2: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #3: 	 Cтатистика = 0.6325 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 5   #4: 	 Cтатистика = 0.9487 t(0.1) = 1.282	 Гипотеза H0 принимается! ( H0 )
Выборка размера 10   #0: 	 Cтатистика = 1.5652 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #1: 	 Cтатистика = 1.3416 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #2: 	 Cтатистика = 2.0125 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #3: 	 Cтатистика = 1.7889 t(0.1) = 1.282	 Гипотеза H0 отклоняется! ( H1 )
Выборка размера 10   #4: 	 Cтатистика = 1.5652 t(0.1) = 1.282	 Гипотеза H0 отклон

На выборках размером 5 все еще наблюдаются ошибки, но с выборками размером 10, мы можем сделать правильный вывод. Это связано с тем что полученное количества материала не точное, а лишь примерное. Судя по результатам, его можно использовать скорее чтобы судить о порядке необходимого числа измерений